In [1]:
from models.model_LIT_CODING import LITIlluminationModel
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from dataset.dataset_utils import SimulatedLabelModule
import torch
import pytorch_lightning as pl

import yaml

In [2]:

photon_count = 10 ** 3
sbr = 1.0

n_tbins = 1000
k = 4
sigma = 10

yaml_file = 'config/best_hyperparameters_tmp.yaml'


In [3]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",  # Directory to save the model
    filename="coded_model",  # Base name for the checkpoint files
    save_top_k=1,  # Save only the best model
    monitor="val_loss",  # Metric to monitor
    mode="min",  # Minimize the monitored metric
)


In [4]:
try:
    with open(yaml_file, 'r') as file:
        config = yaml.safe_load(file)

    init_lr = config['init_lr']
    lr_decay_gamma = config['lr_decay_gamma']
    tv_reg = config['tv_reg']
    epochs = config['epochs']
    batch_size = config['batch_size']
    beta = config['beta']
    num_samples = config['num_samples']
except (FileNotFoundError, TypeError) as e:
    print(e)
    exit(0)


In [5]:
print(tv_reg)

0.00312470677753916


In [6]:
label_module = SimulatedLabelModule(n_tbins, batch_size=batch_size, num_samples=num_samples)
label_module.setup()

print(len(label_module.train_dataset))


67464


In [7]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.set_float32_matmul_precision('medium')

else:
    device = torch.device("cpu")

pl.seed_everything(42)


Seed set to 42


42

In [8]:
logger = CSVLogger("tb_logs", name="my_model")

trainer = pl.Trainer(logger=logger, max_epochs=epochs,
                      log_every_n_steps=250, val_check_interval=0.5,
                      callbacks=[checkpoint_callback])


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [9]:
lit_model = LITIlluminationModel(k=k, n_tbins=n_tbins, init_lr=init_lr, lr_decay_gamma=lr_decay_gamma,
                           beta=beta, tv_reg=tv_reg, photon_count=photon_count, sbr=sbr)

torch.autograd.set_detect_anomaly(True)

In [10]:
trainer.fit(lit_model, datamodule=label_module)

/ua/deparra/miniconda3/envs/learnedcowsip/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /ua/deparra/LearnedCoWSiP/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name         | Type              | Params | Mode 
-----------------------------------------------------------
0 | backbone_net | IlluminationModel | 5.0 K  | train
-----------------------------------------------------------
5.0 K     Trainable params
0         Non-trainable params
5.0 K     Total params
0.020     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode


Epoch 9: 100%|█████████▉| 2498/2499 [01:21<00:00, 30.69it/s, v_num=7]      


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined